# Get Data

## Imports and Setup


In [1]:
# Imports
import requests
from bs4 import BeautifulSoup
from lxml import etree
from time import sleep
import cssselect
import re

In [2]:
# Set constants for use later
BASE_URL = "https://www.mailleartisans.org/weaves/weavedisplay.php?key="
FIRST_ARTICLE = 1
LAST_ARTICLE = 1487

In [3]:

# Create function to get multiple articles
def get_articles(articles) -> dict:
    out = dict()
    for i in articles:
        print(f"Getting article {i}")
        response = requests.get(f"{BASE_URL}{i}")
        if response.status_code == 200:
            out[i] = response.content
            print(f"Got article {i} sleeping for 10 seconds.")
        else:
            out[i] = 'failure'
            print(f"Could not get article {i} sleeping for 10 seconds.")
        sleep(10)
    
    return out


## Test Getting Data From Articles


### Get Test Articles


In [4]:
# Get some articles and store them to avoid repeated calls to the website

to_get = [
    1,    # single AR no Min no Max
    2,    # multi AR no Min no Max
    6,    # single AR no Max
    11,   # multi AR no Max
    189,  # No ARs given
    1086, # Single ar all ARs given
    1173, # multi AR all values given
    1488, # Error page with No AR values
]

arts = get_articles(to_get)



Getting article 1
Got article 1 sleeping for 10 seconds.
Getting article 2
Got article 2 sleeping for 10 seconds.
Getting article 6
Got article 6 sleeping for 10 seconds.
Getting article 11
Got article 11 sleeping for 10 seconds.
Getting article 189
Got article 189 sleeping for 10 seconds.
Getting article 1086
Got article 1086 sleeping for 10 seconds.
Getting article 1173
Got article 1173 sleeping for 10 seconds.
Getting article 1488
Got article 1488 sleeping for 10 seconds.


### Test Single Article

In [26]:
# Select Article
article = arts[1]

# Convert first article into an etree
soup = BeautifulSoup(article, 'html.parser')
dom = etree.HTML(str(soup.html))



# Parse etree to find values of interest

## Determine if an article exists
if len(dom.xpath('/html/body/div[5]/div/h2')) > 0:
        print('No weave here')
else:
    ## Weave Title
    path = '/html/body/div[5]/div/table/tr/td[2]/div[2]/font'
    title = dom.xpath(path)[0].text
    print(f'Weave Title: {title}')

    ## Get AR Values

    ### Get AR val string from page
    path = '/html/body/div[5]/div/table/tr/td[1]/table/tr[2]/td/div[not(@class)][2]//text()'
    ar_val_string = ' '.join([i for i in dom.xpath(path) if i != '\n'][:-1])

    ### Extract max ar value from ar string
    max_ar = re.search('%s(.*)%s' % ('Max. AR :', ' Ideal AR :'), ar_val_string).group(1).replace(' ', '')

    ### Extract ideal ar value from ar string
    ideal_ar = re.search('%s(.*)%s' % (' Ideal AR :', 'Min. AR :'), ar_val_string).group(1).replace(' ', '')

    ### Extract min ar value from ar string
    min_ar = re.search('%s(.*)' % ('Min. AR :'), ar_val_string).group(1).replace(' ', '')

    ### Print ar values from ar string
    print(f'Max AR: {max_ar}, Ideal AR: {ideal_ar}, Min AR: {min_ar}')

    ### Get all dates from page
    date_string = ''.join(results).replace('\n', '')

    ### Get the upload date
    date_uploaded = re.search('%s(.*)%s' % ('Date Uploaded', 'Last Edited'), date_string).group(1).strip()

    ### Get the last edited date
    last_edited = re.search('%s(.*)' % ('Last Edited'), date_string).group(1).strip()

    ### Print out the dates
    print(f'Date Uploaded: {date_uploaded}, Last Edited: {last_edited}')



Weave Title: Trizantine
Max AR: , Ideal AR: 5.2, Min AR: 
Date Uploaded: May 3, 2008, 6:36 pm, Last Edited: June 22, 2017, 11:45 am


### Xpath Testing

In [9]:
# New goal get date data

path = '/html/body/div[5]/div/table/tr/td[1]/table/tr[2]/td/div[not(@class)][3]//text()'

num = 0
results = dom.xpath(path)
result = results[num]

print(f"Current path results: {dom.xpath(path)}")

print(f"Selected Element at path results{result}")

print("Children of selected element at path results:")

for i in result:
    print('\t',i, sep='')


Current path results: ['\n', 'Date Uploaded', '\n              May 3, 2008, 6:36 pm', '\n', 'Last Edited', '\n              June 22, 2017, 11:45 am\n              \n            ']
Selected Element at path results

Children of selected element at path results:
	



### Test All Articles

In [27]:

# Iterate through all articles
for key, value in arts.items():
    print(f'Article: {key}')

    # Convert article to etree
    soup = BeautifulSoup(value, 'html.parser')
    dom = etree.HTML(str(soup.html))

    # Parse etree to find values of interest

    ## Determine if an article exists
    if len(dom.xpath('/html/body/div[5]/div/h2')) > 0:
            print('No weave here')
    else:
        ## Weave Title
        path = '/html/body/div[5]/div/table/tr/td[2]/div[2]/font'
        title = dom.xpath(path)[0].text
        print(f'Weave Title: {title}')

        ## Get AR Values

        ### Get AR val string from page
        path = '/html/body/div[5]/div/table/tr/td[1]/table/tr[2]/td/div[not(@class)][2]//text()'
        ar_val_string = ' '.join([i for i in dom.xpath(path) if i != '\n'][:-1])

        ### Extract max ar value from ar string
        max_ar = re.search('%s(.*)%s' % ('Max. AR :', ' Ideal AR :'), ar_val_string).group(1).replace(' ', '')

        ### Extract ideal ar value from ar string
        ideal_ar = re.search('%s(.*)%s' % (' Ideal AR :', 'Min. AR :'), ar_val_string).group(1).replace(' ', '')

        ### Extract min ar value from ar string
        min_ar = re.search('%s(.*)' % ('Min. AR :'), ar_val_string).group(1).replace(' ', '')

        ### Print ar values from ar string
        print(f'Max AR: {max_ar}, Ideal AR: {ideal_ar}, Min AR: {min_ar}')

        ### Get all dates from page
        date_string = ''.join(results).replace('\n', '')

        ### Get the upload date
        date_uploaded = re.search('%s(.*)%s' % ('Date Uploaded', 'Last Edited'), date_string).group(1).strip()

        ### Get the last edited date
        last_edited = re.search('%s(.*)' % ('Last Edited'), date_string).group(1).strip()

        ### Print out the dates
        print(f'Date Uploaded: {date_uploaded}, Last Edited: {last_edited}')
    
    print()



Article: 1
Weave Title: Trizantine
Max AR: , Ideal AR: 5.2, Min AR: 
Date Uploaded: May 3, 2008, 6:36 pm, Last Edited: June 22, 2017, 11:45 am

Article: 2
Weave Title: Hizashi 1
Max AR: , Ideal AR: 8.0|3.0, Min AR: 
Date Uploaded: May 3, 2008, 6:36 pm, Last Edited: June 22, 2017, 11:45 am

Article: 6
Weave Title: European 4 in 1
Max AR: , Ideal AR: 4.0, Min AR: 2.83
Date Uploaded: May 3, 2008, 6:36 pm, Last Edited: June 22, 2017, 11:45 am

Article: 11
Weave Title: Dragonscale
Max AR: , Ideal AR: 3.9|6.1, Min AR: 3.7|5.7
Date Uploaded: May 3, 2008, 6:36 pm, Last Edited: June 22, 2017, 11:45 am

Article: 189
Weave Title: Captive Persian Round Sheet
Max AR: , Ideal AR: , Min AR: 
Date Uploaded: May 3, 2008, 6:36 pm, Last Edited: June 22, 2017, 11:45 am

Article: 1086
Weave Title: Cloudy Day
Max AR: 4.5, Ideal AR: 3.9, Min AR: 3.6
Date Uploaded: May 3, 2008, 6:36 pm, Last Edited: June 22, 2017, 11:45 am

Article: 1173
Weave Title: Onering
Max AR: 3.0|6.9, Ideal AR: 2.9|5.0, Min AR: 2.7|3.0